# PolynomialCompressor class

- Off-the-shelf compressors for refrigeration machines or heat pumps
- Use datasheet to compute polynomials for isentropic and volumetric efficiency
  as function of evaporation and condensation temperature (DIN EN12900)